In [2]:
import sys
sys.executable
import os
os.getcwd()
sys.path.append('devel/lib/python3/dist-packages/theconstruct_msgs/msg/')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src/moveo_training/scripts')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages/openai_ros/msg')

In [3]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf
from src.moveo_training.src.moveo_training import moveo_inverse_kinematic
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import py_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils
import rospy
from tf_agents.environments import suite_gym
from tf_agents.train.utils import strategy_utils
tempdir = tempfile.gettempdir()

2021-08-31 16:14:12.392070: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2021-08-31 16:14:12.392104: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [4]:
rospy.init_node("train_moveo")
env_name = 'MoveoIK-v0' # @param {type:"string"}
env = suite_gym.load(env_name)

[ERROR] [1630419256.130143, 58.384000]: NOT Initialising Simulation Physics Parameters
[WARN] [1630419256.135388, 58.389000]: Start Init ControllersConnection
[WARN] [1630419256.136725, 58.389000]: END Init ControllersConnection
/home/nils/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
policy_dir ="/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/policies/collect_policy"
eager_py_policy = py_tf_eager_policy.SavedModelPyTFEagerPolicy(
    policy_dir, env.time_step_spec(), env.action_spec())


2021-08-31 16:14:16.335936: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-31 16:14:16.368459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 16:14:16.368736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 970 computeCapability: 5.2
coreClock: 1.2155GHz coreCount: 13 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-31 16:14:16.368851: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros

In [6]:

# eager_py_policy.action(time_step)

In [7]:
time_step = env.reset()
initial_collect_actor = actor.Actor(
  env,
  eager_py_policy,
  time_step,
  steps_per_run=1,
  )
initial_collect_actor.run()


2021-08-31 16:14:17.594770: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-31 16:14:17.612106: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3192625000 Hz


In [13]:

initial_collect_actor.run()

[ WARN] [1630419358.010440915, 157.235000000]: Fail: ABORTED: No motion plan found. No execution attempted.


In [15]:
from tf_agents.utils import common
checkpoint_dir = "/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/policies/collect_policy"
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    # agent=agent,
    # policy=agent.policy,
    # replay_buffer=replay_buffer,
    # global_step=global_step
)


In [20]:
train_checkpointer.initialize_or_restore()
# global_step = tf.compat.v1.train.get_global_step()
# collect_actor.run()

shutdown request: [/train_moveo] Reason: new node registered with same name
